In [31]:
import os
import torch
from pytorch_lightning import LightningModule, Trainer, seed_everything
import tensorflow as tf
import transformers
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

import re
import emoji
from soynlp.normalizer import repeat_normalize

In [32]:
!git clone https://github.com/e9t/nsmc.git

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [33]:
!ls nsmc -la

total 38632
drwxr-xr-x  5 aiffel-dj57 aiffel-dj57     4096  5월 22 17:15 .
drwxr-xr-x 70 aiffel-dj57 aiffel-dj57     4096  5월 23 15:53 ..
drwxr-xr-x  2 aiffel-dj57 aiffel-dj57     4096  5월 22 17:15 code
drwxr-xr-x  8 aiffel-dj57 aiffel-dj57     4096  5월 22 17:15 .git
-rw-r--r--  1 aiffel-dj57 aiffel-dj57  4893335  5월 22 17:15 ratings_test.txt
-rw-r--r--  1 aiffel-dj57 aiffel-dj57 14628807  5월 22 17:15 ratings_train.txt
-rw-r--r--  1 aiffel-dj57 aiffel-dj57 19515078  5월 22 17:15 ratings.txt
drwxr-xr-x  2 aiffel-dj57 aiffel-dj57   458752  5월 22 17:15 raw
-rw-r--r--  1 aiffel-dj57 aiffel-dj57     2596  5월 22 17:15 README.md
-rw-r--r--  1 aiffel-dj57 aiffel-dj57    36746  5월 22 17:15 synopses.json


In [34]:
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)
train.head(5)

(150000, 3)
(50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 1. BERT-BASE-MULTLINGUAL PRETRAINED MODEL

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

In [6]:
# BERT의 입력 형식에 맞게 토큰을 추가해줍니다
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in train.document]
sentences[:10]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]',
 '[CLS] 막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움. [SEP]',
 '[CLS] 원작의 긴장감을 제대로 살려내지못했다. [SEP]',
 '[CLS] 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네 [SEP]',
 '[CLS] 액션이 없는데도 재미 있는 몇안되는 영화 [SEP]',
 '[CLS] 왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? [SEP]']

In [7]:
#토큰화 확인
lines = [
  "아이펠 정책 피드백 감정분석팀 '정check'입니다.",
  "구글 리뷰 데이터와 네이버 블로그 데이터를 사용해서 감정분석을 실행할 예정입니다.",
  "열심히 하겠습니다."
]
encoded_lines2 = tokenizer(lines)
print(encoded_lines2)

{'input_ids': [[101, 9519, 10739, 119394, 9670, 119254, 9946, 15001, 63218, 8848, 16605, 37712, 40958, 74399, 112, 9670, 11119, 11263, 112, 9645, 48345, 119, 102], [101, 8908, 118663, 9238, 118999, 9083, 85297, 12638, 9011, 10739, 41605, 9378, 11261, 78136, 9083, 85297, 11513, 9405, 24974, 70146, 8848, 16605, 37712, 40958, 10622, 9489, 25549, 14843, 9576, 16605, 58303, 48345, 119, 102], [101, 9569, 71013, 18108, 9952, 118632, 119081, 48345, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [8]:
tokenized_texts1 = [tokenizer.tokenize(str(lines)) for line in lines]
print(tokenized_texts1[:3]) 

[['[', '"', '아', '##이', '##펠', '정', '##책', '피', '##드', '##백', '감', '##정', '##분', '##석', '##팀', "'", '정', '##che', '##ck', "'", '입', '##니다', '.', '"', ',', "'", '구', '##글', '리', '##뷰', '데', '##이터', '##와', '네', '##이', '##버', '블', '##로', '##그', '데', '##이터', '##를', '사', '##용', '##해서', '감', '##정', '##분', '##석', '##을', '실', '##행', '##할', '예', '##정', '##입', '##니다', '.', "'", ',', "'", '열', '##심', '##히', '하', '##겠', '##습', '##니다', '.', "'", ']'], ['[', '"', '아', '##이', '##펠', '정', '##책', '피', '##드', '##백', '감', '##정', '##분', '##석', '##팀', "'", '정', '##che', '##ck', "'", '입', '##니다', '.', '"', ',', "'", '구', '##글', '리', '##뷰', '데', '##이터', '##와', '네', '##이', '##버', '블', '##로', '##그', '데', '##이터', '##를', '사', '##용', '##해서', '감', '##정', '##분', '##석', '##을', '실', '##행', '##할', '예', '##정', '##입', '##니다', '.', "'", ',', "'", '열', '##심', '##히', '하', '##겠', '##습', '##니다', '.', "'", ']'], ['[', '"', '아', '##이', '##펠', '정', '##책', '피', '##드', '##백', '감', '##정', '##분', '##석', '##팀', "'", '정', '##che', '#

In [9]:
#로드한 토크나이저로 토큰화
tokenized_texts = [tokenizer.tokenize(sentence) for sentence in sentences]
print(tokenized_texts[:3])

[['[CLS]', '아', '더', '##빙', '.', '.', '진', '##짜', '짜', '##증', '##나', '##네', '##요', '목', '##소', '##리', '[SEP]'], ['[CLS]', '[UNK]', '.', '.', '.', '포', '##스터', '##보', '##고', '초', '##딩', '##영', '##화', '##줄', '.', '.', '.', '.', '오', '##버', '##연', '##기', '##조', '##차', '가', '##볍', '##지', '않', '##구', '##나', '[SEP]'], ['[CLS]', '[UNK]', '[SEP]']]


In [10]:
#BERT의 단어 집합에 특정 단어가 있는지 조회
print(tokenizer.vocab['감정'])
print(tokenizer.vocab['블로그']) 
#key_error : bert 단어 집합에 존재하지 않는 단어. multilingual language model을 불러왔는데 기본적인 '감정'이 없다는게 흥미롭다

KeyError: '감정'

In [11]:
print(tokenizer.vocab['sentiment']) #영어로 치니까 나오네요.

70762


In [12]:
#패딩
#token들의 max length보다 크게 MAX_LEN을 설정합니다.
#설정한 MAX_LEN 만큼 빈 공간을 0이 채웁니다.
MAX_LEN = 128 
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]

array([   101,   9519,   9074, 119005,    119,    119,   9708, 119235,
         9715, 119230,  16439,  77884,  48549,   9284,  22333,  12692,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [13]:
#어텐션 마스크 : 실제 단어와 패딩 토큰을 구분. 패딩의 위치는 0 실단어의 위치는 1로 구분
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [14]:
#train_test_validation split
train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, train['label'].values, random_state=2, test_size=0.2)

#어텐션 마스크 또한 test와 validation으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2, 
                                                       test_size=0.2)

In [15]:
# 데이터를 파이토치의 텐서로 변환
import torch
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [16]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


batch_size = 32 #batch size 62로 실험해보았으나 메모리 에러로 32로 고정

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
#pytorch dataloader
#PyTorch는 torch.utils.data.Dataset으로 Custom Dataset을 만든다
#torch.utils.data.DataLoader로는 데이터를 불러온다.

#TensorDataset : tensor를 감싸는 Dataset


#pytorch sampler
#sampler : index를 컨트롤하는 방법 / SequentialSampler : 인덱스를 항상 같은 순서로 불러온다  
#참고 : https://subinium.github.io/pytorch-dataloader/

In [17]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2070


In [18]:
# Outputs of BERT, corresponding to one output vector of size 768 for each input token
#outputs = model(input_ids,
#                    attention_mask=attention_mask,
#                    token_type_ids=token_type_ids,
#                    position_ids=position_ids, 
#                    head_mask=head_mask)

# Grab the [CLS] token, used as an aggregate output representation for classification tasks
#pooled_output = outputs[1]

# Create dropout (for training)
#dropout = nn.Dropout(hidden_dropout_prob)

# Apply dropout between the output of the BERT transformers and our final classification layer
#pooled_output = dropout(pooled_output)

# Create our classification layer
#classifier = nn.Linear(hidden_size, num_labels)

# Feed the pooled output through the classifier 
#logits = classifier(pooled_output)

In [19]:
#bert 모델 생성

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()
#num_labels=2 => 이진분류
#BertForSequenceClassification을 사용해서 모델구조를 불러올 수 있다.


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [20]:
#Huggning Face의 BertForSequenceClassification() 함수를 사용해서  파인튜닝 진행
#Linear classifier(선형분류) 중 binary classification을 진행해야함
#(classifier): Linear(in_features=768, out_features=2, bias=True)

In [21]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [22]:
#모델학습
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [23]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [24]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  3,750.    Elapsed: 0:04:12.
  Batch 1,000  of  3,750.    Elapsed: 0:08:22.
  Batch 1,500  of  3,750.    Elapsed: 0:12:27.
  Batch 2,000  of  3,750.    Elapsed: 0:16:31.
  Batch 2,500  of  3,750.    Elapsed: 0:20:35.
  Batch 3,000  of  3,750.    Elapsed: 0:24:40.
  Batch 3,500  of  3,750.    Elapsed: 0:28:44.

  Average training loss: 0.39
  Training epcoh took: 0:30:46

Running Validation...
  Accuracy: 0.85
  Validation took: 0:02:25

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  3,750.    Elapsed: 0:04:04.
  Batch 1,000  of  3,750.    Elapsed: 0:08:09.
  Batch 1,500  of  3,750.    Elapsed: 0:12:13.
  Batch 2,000  of  3,750.    Elapsed: 0:16:17.
  Batch 2,500  of  3,750.    Elapsed: 0:20:21.
  Batch 3,000  of  3,750.    Elapsed: 0:24:25.
  Batch 3,500  of  3,750.    Elapsed: 0:28:30.

  Average training loss: 0.29
  Training epcoh took: 0:30:32

Running Validation...
  Accuracy: 0.87
  Validation took: 0:02:2

In [ ]:
#barch size가 64면 out of memory 문제 발생

In [27]:
#test 전처리밑 데이터로더 형성
BATCH_SIZE = 32

sentences = test['document']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = test['label'].values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [28]:
#test data에 확인
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,563.    Elapsed: 0:00:15.
  Batch   200  of  1,563.    Elapsed: 0:00:31.
  Batch   300  of  1,563.    Elapsed: 0:00:46.
  Batch   400  of  1,563.    Elapsed: 0:01:02.
  Batch   500  of  1,563.    Elapsed: 0:01:17.
  Batch   600  of  1,563.    Elapsed: 0:01:33.
  Batch   700  of  1,563.    Elapsed: 0:01:48.
  Batch   800  of  1,563.    Elapsed: 0:02:04.
  Batch   900  of  1,563.    Elapsed: 0:02:19.
  Batch 1,000  of  1,563.    Elapsed: 0:02:35.
  Batch 1,100  of  1,563.    Elapsed: 0:02:51.
  Batch 1,200  of  1,563.    Elapsed: 0:03:07.
  Batch 1,300  of  1,563.    Elapsed: 0:03:23.
  Batch 1,400  of  1,563.    Elapsed: 0:03:39.
  Batch 1,500  of  1,563.    Elapsed: 0:03:55.

Accuracy: 0.87
Test took: 0:04:05
